In [1]:
!pip uninstall -y torch torchtext
!pip install torch==2.2.2 torchtext==0.17.2

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
 

In [ ]:
pip install --upgrade safetensors transformers

In [2]:
# importing required libraries
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import torchtext
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
print(torch.__version__)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

2.2.2+cu121


In [20]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        """
        Args:
            vocab_size: size of vocabulary
            embed_dim: dimension of embeddings
        """
        super(Embedding, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            out: embedding vector
        """
        out = self.embed(x)
        return out

In [21]:
class PositionalEmbedding(nn.Module):
    def __init__(self, max_seq_len, embed_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: emdedding dimension
        """
        super(PositionalEmbedding, self).__init__()
        self.embed_model_dim = embed_model_dim

        position_embed = torch.zeros(max_seq_len, embed_model_dim)
        for pos in range(max_seq_len):
          for i in range(embed_model_dim):
            if(i%2):
              position_embed[pos,i] = math.sin(pos/(10000**((2*i)/embed_model_dim)))
            else:
              position_embed[pos,i] = math.cos(pos/(10000**((2*i)/embed_model_dim)))
        position_embed = position_embed.unsqueeze(0)
        self.register_buffer('position_embed', position_embed)
    def forward(self, x):
      x = x*math.sqrt(self.embed_model_dim)
      seq_len = x.size(1)
      out = x + torch.autograd.Variable(self.position_embed[:, :seq_len], requires_grad=False)
      return out

In [22]:
class MultiHeadAttention(nn.Module):
  def __init__(self, embed_dim, num_heads):
    super(MultiHeadAttention, self).__init__()

    self.embed_dim = embed_dim    #512 dim
    self.n_heads = num_heads   #8
    self.single_head_dim = int(self.embed_dim/self.n_heads)
    self.key = nn.Linear(self.single_head_dim, self.single_head_dim, bias=False)
    self.query = nn.Linear(self.single_head_dim, self.single_head_dim, bias=False)
    self.value = nn.Linear(self.single_head_dim, self.single_head_dim, bias=False)
    self.out = nn.Linear(self.n_heads*self.single_head_dim, self.embed_dim)

  def forward(self, key, query, value, mask=None):
    batch_size = key.size(0)
    seq_len = key.size(1)

    # query dimension can change in decoder during inference.
    # so we cant take general seq_length
    seq_length_query = query.size(1)

    key = key.view(batch_size, seq_len, self.n_heads, self.single_head_dim)
    query = query.view(batch_size, seq_length_query, self.n_heads, self.single_head_dim)
    value = value.view(batch_size, seq_len, self.n_heads, self.single_head_dim)
    k = self.key(key)
    q = self.query(query)
    v = self.value(value)

    k = k.transpose(1,2)
    q = q.transpose(1,2)
    v = v.transpose(1,2)

    # adjust key for matrix multiplication
    k_adjusted = k.transpose(-1,-2)  #(batch_size, n_heads, single_head_dim, seq_ken)  #(32 x 8 x 64 x 10)
    product = torch.matmul(q, k_adjusted)  #(batch_size, n_heads, seq_len_q, seq_len_k)  #(32 x 8 x 10 x 10)

    # fill those positions of product matrix as (-1e20) where mask positions are 0
    if mask is not None:
      product = product.masked_fill(mask == 0, float("-1e20"))

    product = product / math.sqrt(self.single_head_dim) #(32 x 8 x 10 x 64) x (32 x 8 x 64 x 10) = #(32x8x10x10)

    scores = F.softmax(product, dim=-1)

    scores = torch.matmul(scores, v)##(32x8x 10x 10) x (32 x 8 x 10 x 64) = (32 x 8 x 10 x 64)

    #concatenated output
    concat = scores.transpose(1,2).contiguous().view(batch_size, seq_length_query, self.single_head_dim*self.n_heads) # (32x8x10x64) -> (32x10x8x64)  -> (32,10,512)

    output = self.out(concat) #(32,10,512) -> (32,10,512)

    return output

In [23]:
class TransformerBlock(nn.Module):
  def __init__(self, embed_dim, num_heads, dropout, forward_expansion):
    super(TransformerBlock, self).__init__()
    self.attention = MultiHeadAttention(embed_dim, num_heads)
    self.norm1 = nn.LayerNorm(embed_dim)
    self.norm2 = nn.LayerNorm(embed_dim)

    self.feed_forward = nn.Sequential(
        nn.Linear(embed_dim, forward_expansion*embed_dim),
        nn.ReLU(),
        nn.Linear(forward_expansion*embed_dim, embed_dim)
    )

    self.dropout = nn.Dropout(dropout)

  def forward(self, key, query, value):
    attention = self.attention(key, query, value)
    norm1_out = self.dropout(self.norm1(attention + query))
    feed_forward_out = self.feed_forward(norm1_out)
    norm2_out = self.dropout(self.norm2(feed_forward_out + norm1_out))
    return norm2_out


In [24]:
class TransformerEncoder(nn.Module):
  def __init__(self, embed_dim, num_heads, dropout, forward_expansion, max_seq_len, num_layers, vocab_size):
    super(TransformerEncoder, self).__init__()
    self.embedding_layer = Embedding(vocab_size, embed_dim)
    self.positional_embedding = PositionalEmbedding(max_seq_len, embed_dim)
    self.layers = nn.ModuleList([TransformerBlock(embed_dim, num_heads, dropout, forward_expansion) for _ in range(num_layers)])

  def forward(self, x):
    embed_out = self.embedding_layer(x)
    out = self.positional_embedding(embed_out)
    for layer in self.layers:
      out = layer(out, out, out)
    return out

In [30]:
class DecoderBlock(nn.Module):
  def __init__(self, embed_dim, num_heads, dropout, forward_expansion):
    super(DecoderBlock, self).__init__()
    self.attention = MultiHeadAttention(embed_dim, num_heads)
    self.norm = nn.LayerNorm(embed_dim)
    self.dropout = nn.Dropout(dropout)
    self.transformer_block = TransformerBlock(embed_dim, num_heads, dropout, forward_expansion)

  def forward(self, enc_out, x, trg_mask):
        # 1️⃣ Masked self-attention (Q=K=V from decoder input x)
        self_attn_out = self.attention(x, x, x, trg_mask)
        x = self.dropout(self.norm(self_attn_out + x))

        # 2️⃣ Cross-attention (Q from decoder, K=V from encoder) — no trg_mask here!
        out = self.transformer_block(enc_out, x, enc_out)
        return out

In [31]:
class TransformerDecoder(nn.Module):
  def __init__(self, embed_dim, num_heads, dropout, forward_expansion, max_seq_len, num_layers, vocab_size):
    super(TransformerDecoder, self).__init__()
    self.embedding_layer = Embedding(vocab_size, embed_dim)
    self.positional_embedding = PositionalEmbedding(max_seq_len, embed_dim)
    self.layers = nn.ModuleList([DecoderBlock(embed_dim, num_heads, dropout, forward_expansion) for _ in range(num_layers)])
    self.ff = nn.Linear(embed_dim, vocab_size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, enc_out, trg_mask):
    embed_out = self.embedding_layer(x)
    out = self.positional_embedding(embed_out)
    out = self.dropout(out)

    for layer in self.layers:
      out = layer(enc_out, out, trg_mask)
    out = self.ff(out)
    return out

In [38]:
class Transformer(nn.Module):
  def __init__(self, embed_dim, src_vocab_size, trg_vocab_size, num_heads, dropout, forward_expansion, max_seq_len, num_layers):
    super(Transformer, self).__init__()
    self.encoder = TransformerEncoder(embed_dim, num_heads, dropout, forward_expansion, max_seq_len, num_layers, src_vocab_size)
    self.decoder = TransformerDecoder(embed_dim, num_heads, dropout, forward_expansion, max_seq_len, num_layers, trg_vocab_size)

  def make_trg_mask(self, trg):
    batch_size, trg_seq_len = trg.shape
    mask = torch.tril(torch.ones((trg_seq_len, trg_seq_len), device=trg.device)).bool()
    return mask.unsqueeze(0).unsqueeze(0).expand(batch_size, 1, trg_seq_len, trg_seq_len)

  def decode(self, src, trg):
    """
        for inference
        Args:
            src: input to encoder
            trg: input to decoder
        out:
            out_labels : returns final prediction of sequence
    """
    trg_mask = self.make_trg_mask(trg)
    enc_out = self.encoder(src)
    output = []
    batch_size,seq_len = src.shape[0],src.shape[1]
    out = trg
    for i in range(seq_len):
      out = self.decoder(out, enc_out, trg_mask)
      out = out[:,-1:,:]
      out = out.argmax(-1)
      print(out)
      output.append(out.item())
      out = torch.unsqueeze(out,axis=0)

    return output

  def forward(self, src, trg):
        """
        Args:
            src: input to encoder
            trg: input to decoder
        out:
            out: final vector which returns probabilities of each target word
        """
        trg_mask = self.make_trg_mask(trg)
        enc_out = self.encoder(src)

        outputs = self.decoder(trg, enc_out, trg_mask)
        return outputs


In [33]:
src_vocab_size = 11
target_vocab_size = 11
num_layers = 6
seq_length= 12


# let 0 be sos token and 1 be eos token
src = torch.tensor([[0, 2, 5, 6, 4, 3, 9, 5, 2, 9, 10, 1],
                    [0, 2, 8, 7, 3, 4, 5, 6, 7, 2, 10, 1]])
target = torch.tensor([[0, 1, 7, 4, 3, 5, 9, 2, 8, 10, 9, 1],
                       [0, 1, 5, 6, 2, 4, 7, 6, 2, 8, 10, 1]])

print(src.shape,target.shape)
model = Transformer(embed_dim=512, src_vocab_size=src_vocab_size,
                    trg_vocab_size=target_vocab_size, max_seq_len=seq_length,
                    num_layers=num_layers, forward_expansion=4, num_heads=8, dropout= 0.2)
model

torch.Size([2, 12]) torch.Size([2, 12])


Transformer(
  (encoder): TransformerEncoder(
    (embedding_layer): Embedding(
      (embed): Embedding(11, 512)
    )
    (positional_embedding): PositionalEmbedding()
    (layers): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadAttention(
          (key): Linear(in_features=64, out_features=64, bias=False)
          (query): Linear(in_features=64, out_features=64, bias=False)
          (value): Linear(in_features=64, out_features=64, bias=False)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (feed_forward): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (decoder): Transform

In [34]:
out = model(src, target)
out.shape

torch.Size([2, 12, 11])

In [40]:
# inference
model = Transformer(embed_dim=512, src_vocab_size=src_vocab_size,
                    trg_vocab_size=target_vocab_size, max_seq_len=seq_length,
                    num_layers=num_layers, forward_expansion=4, num_heads=8, dropout= 0.2)



src = torch.tensor([[0, 2, 5, 6, 4, 3, 9, 5, 2, 9, 10, 1]])
trg = torch.tensor([[0]])
print(src.shape,trg.shape)
out = model.decode(src, trg)
out

torch.Size([1, 12]) torch.Size([1, 1])
tensor([[6]])
tensor([[[3]]])
tensor([[[[1]]]])
tensor([[[[[4]]]]])
tensor([[[[[[4]]]]]])
tensor([[[[[[[3]]]]]]])
tensor([[[[[[[[2]]]]]]]])
tensor([[[[[[[[[3]]]]]]]]])
tensor([[[[[[[[[[2]]]]]]]]]])
tensor([[[[[[[[[[[3]]]]]]]]]]])
tensor([[[[[[[[[[[[6]]]]]]]]]]]])
tensor([[[[[[[[[[[[[6]]]]]]]]]]]]])


[6, 3, 1, 4, 4, 3, 2, 3, 2, 3, 6, 6]

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim

# ----- Hyperparameters -----
src_vocab_size = 11
trg_vocab_size = 11
embed_dim = 512
num_heads = 8
num_layers = 6
dropout = 0.2
forward_expansion = 4
max_seq_len = 12
learning_rate = 1e-4
num_epochs = 200

# ----- Model, Loss, Optimizer -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(embed_dim, src_vocab_size, trg_vocab_size, num_heads, dropout, forward_expansion, max_seq_len, num_layers).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)  # ignoring padding index if 0 is pad token
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ----- Dummy Training Data (replace with real data) -----
src = torch.tensor([[0, 2, 5, 6, 4, 3, 9, 5, 2, 9, 10, 1],
                    [0, 2, 8, 7, 3, 4, 5, 6, 7, 2, 10, 1]]).to(device)

trg = torch.tensor([[0, 1, 7, 4, 3, 5, 9, 2, 8, 10, 9, 1],
                    [0, 1, 5, 6, 2, 4, 7, 6, 2, 8, 10, 1]]).to(device)

for epoch in range(num_epochs):
    model.train()

    # Decoder input = all except last token of target
    dec_input = trg[:, :-1]
    # Target for loss = all except first token of target
    dec_target = trg[:, 1:]

    output = model(src, dec_input)  # shape: (batch, seq_len-1, vocab_size)

    # Reshape for loss
    output = output.reshape(-1, output.shape[-1])   # (batch*(seq_len-1), vocab_size)
    dec_target = dec_target.reshape(-1)             # (batch*(seq_len-1))

    loss = criterion(output, dec_target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]  Loss: {loss.item():.4f}")

print("Training complete ✅")

Epoch [10/200]  Loss: 2.2254
Epoch [20/200]  Loss: 2.2335
Epoch [30/200]  Loss: 2.0626
Epoch [40/200]  Loss: 2.3020
Epoch [50/200]  Loss: 2.0602
Epoch [60/200]  Loss: 1.9241
Epoch [70/200]  Loss: 1.7000
Epoch [80/200]  Loss: 1.5404
Epoch [90/200]  Loss: 1.3410
Epoch [100/200]  Loss: 1.0879
Epoch [110/200]  Loss: 0.9519
Epoch [120/200]  Loss: 0.6945
Epoch [130/200]  Loss: 0.5535
Epoch [140/200]  Loss: 0.4356
Epoch [150/200]  Loss: 0.3458
Epoch [160/200]  Loss: 0.4664
Epoch [170/200]  Loss: 0.3732
Epoch [180/200]  Loss: 0.2881
Epoch [190/200]  Loss: 0.3040
Epoch [200/200]  Loss: 0.2971
Training complete ✅


In [41]:
src = torch.tensor([[0, 2, 5, 6, 4, 3, 9, 5, 2, 9, 10, 1]])
trg = torch.tensor([[0]])
print(src.shape,trg.shape)
out = model.decode(src, trg)
out

torch.Size([1, 12]) torch.Size([1, 1])
tensor([[8]])
tensor([[[10]]])
tensor([[[[4]]]])
tensor([[[[[0]]]]])
tensor([[[[[[4]]]]]])
tensor([[[[[[[10]]]]]]])
tensor([[[[[[[[4]]]]]]]])
tensor([[[[[[[[[4]]]]]]]]])
tensor([[[[[[[[[[4]]]]]]]]]])
tensor([[[[[[[[[[[0]]]]]]]]]]])
tensor([[[[[[[[[[[[3]]]]]]]]]]]])
tensor([[[[[[[[[[[[[9]]]]]]]]]]]]])


[8, 10, 4, 0, 4, 10, 4, 4, 4, 0, 3, 9]